In [1]:
from graph.process.tools import make_sql_tool, make_rag_tool
from dotenv import load_dotenv
from pathlib import Path

import os
env_path = Path("..")/".env"
load_dotenv(env_path)

True

In [2]:
config = {"llm": {
        "model": "ollama:qwen3:8b",
        "temperature": 0.0
      }}
validation_config = {}
sql_tool = make_sql_tool(config, validation_config)

In [3]:
ret = sql_tool.invoke("how many steps did i take last week?")
ret

'You took a total of 86,583 steps last week (April 4th to April 10th, 2016).'

In [3]:
from graph.process.tools import make_rag_tool

config = {
      "query_rewriter": {
        "llm": {
          "model": "ollama:qwen3:8b",
          "temperature": 0.0
        }
      },
      "retriever": {
        "embeddings": {
          "provider": "ollama",
          "model": "mxbai-embed-large:335m",
        },
        "retriever_k": 3,
        "score_threshold": 0.0
      },
      "generate": {
        "llm": {
          "model": "ollama:qwen3:8b",
          "temperature": 0.0
        },
        "prompt_repo": "rlm/rag-prompt"
      },
      "grade_documents": {
        "min_relevance_score": 0.6,
        "llm": {
          "model": "ollama:gemma3:270m",
          "temperature": 0.0
        }
      }
    }


rag_tool = make_rag_tool(config)

In [4]:
ret = rag_tool.invoke("How much water should i drink during the day?")
ret

---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GENERATE---


'The Mayo Clinic recommends about 3.7 liters (15.5 cups) for men and 2.7 liters (11.5 cups) for women daily, with 20% of fluid intake typically from food. Adjust based on activity, climate, pregnancy, or illness. Drink water regularly throughout the day rather than waiting for thirst.'

In [9]:
from langchain_core.messages import SystemMessage, HumanMessage
from graph.process.prompts.execution import EXECUTION_PROMPT
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
# 1. Instantiate
llm = init_chat_model()

agent = create_agent(
    model=llm,
    tools=[fetch_knowledge_base],
    system_prompt=EXECUTION_PROMPT.format(process_plan="FETCH WATER INTAKE DATA")
)



In [11]:
inputs = {"messages": [HumanMessage(content="How much water should I drink daily?")]}
response_state = agent.invoke(inputs)

print(response.tool_calls)

---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GENERATE---
[{'name': 'fetch_knowledge_base', 'args': {'question': 'How much water should a person drink daily? What are the recommended daily water intake guidelines?'}, 'id': 'toolu_0114tT6VbFcN2yJiB3kVnVgB', 'type': 'tool_call'}]


In [12]:
response_state


{'messages': [HumanMessage(content='How much water should I drink daily?', additional_kwargs={}, response_metadata={}, id='4a8f5c23-126a-4691-b92e-2156c09d9970'),
  AIMessage(content=[{'text': '**Thought**: The user is asking about daily water intake recommendations. The plan indicates I should "FETCH WATER INTAKE DATA," but this appears to be asking for general health guidance about recommended water consumption rather than the user\'s personal water intake data. \n\nLooking at my available tools:\n- `sql_data_tool`: Accesses user health metrics (steps, heart rate, sleep) - but water intake is not mentioned as available\n- `kb_rag_tool` (fetch_knowledge_base): Accesses scientific knowledge or behavioral research - this would be appropriate for general water intake recommendations\n\nI should use the knowledge base tool to get scientifically-backed information about daily water intake recommendations.\n\n**Action**: I\'ll query the knowledge base for information about daily water intak

In [ ]:
type(res)